# Laser sentence Embeddings to Qwen-math token constructor

In [ ]:
%pip install laser_encoders

# Imports

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from datasets import load_dataset

In [ ]:
from laser_encoders import LaserEncoderPipeline

In [ ]:
from tqdm import tqdm

In [ ]:
import re

# Matrix Multiplication Configurations

In [ ]:
# Used for high precision matrix multiplication in GPUs like A100
# Comment out if not using supported GPU
torch.set_float32_matmul_precision('high')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
number_of_gpus = torch.cuda.device_count()

# Parameters

In [ ]:
cache_dir = "data_cache"
model_dir = "model_cache"

In [ ]:
batch_size = 1024

In [ ]:
max_tokens = 100

In [ ]:
max_gpus = 4  
max_gpus = min(max_gpus, number_of_gpus)
gpus = list(range(max_gpus))

# Dataset Loading and Preprocessing

In [ ]:
def extract_num_output(text):
    match = re.search(r'(?<=The answer is:\s).*$', text)
    if match:
        return match.group(0)
    return None

In [ ]:
meta_math_ds = load_dataset("meta-math/MetaMathQA", cache_dir=cache_dir)
# Sentence dataset is used to get more english data
sen_ds = load_dataset("sentence-transformers/wikipedia-en-sentences",cache_dir=cache_dir)

In [ ]:
meta_math_df = meta_math_ds['train'].to_pandas()
sen_df = sen_ds['train'].to_pandas()

In [ ]:
meta_math_df['num_output'] = meta_math_df['output'].apply(extract_num_output)

# Model loading